# Pub/Sub

- 참고 (2018년 국문 번역문) : https://blog.storyg.co/rabbitmqs/tutorials/python/04-routing

- 참고 (최신 정식 문서) : https://www.rabbitmq.com/tutorials/tutorial-three-python.html

- 참고 : https://github.com/rabbitmq/rabbitmq-tutorials


- 본 문의 표현 및 글은 상기 블로그의 내용을 거의 인용했습니다.

- RabbitMQ가 표준 포트 (5672)의 localhost에 설치되어 실행되고 있다고 가정합니다. 다른 호스트, 포트 또는 자격 증명을 사용하는 경우 연결 설정을 조정해야 합니다.
- 이번에는 여러 소비자(컨슈머)에게 메시지를 전달할 것입니다. 

## 핵심 요소 
- 프로듀서(생성자)는 메시지를 보내는 사용자 응용 프로그램입니다.
- 큐는 메시지를 저장하는 버퍼입니다.
- 컨슈머(소비자)는 메시지를받는 사용자 응용 프로그램입니다.

## exchange vs. queue
- RabbitMQ의 메시징 모델에서 핵심 아이디어는 프로듀서가 어떤 메시지도 큐로 직접 보내지 않는다는 것입니다. 사실, 프로듀서는 메시지가 어떤 큐에 전달되는지 전혀 모릅니다.
- 대신 프로듀서는 메시지를 exchange에만 보낼 수 있습니다. exchange는 아주 간단한 일입니다. 한 쪽에서는 프로듀서로부터 메시지를 받고 다른 쪽에서는 메시지를 큐로 보냅니다. exchange는 받은 메시지를 어떻게 처리해야 하는지 정확히 알아야 합니다.

  (1) 특정 큐에 추가해야 하나?
  
  (2) 여러 큐에 추가해야 하나?
  
  (3) 폐기해야 하나?

In [1]:
! rabbitmqctl --version

3.10.7


In [2]:
! rabbitmqctl list_exchanges

Listing exchanges for vhost / ...
name	type
amq.headers	headers
amq.match	headers
amq.rabbitmq.trace	topic
amq.direct	direct
amq.topic	topic
amq.fanout	fanout
	direct


In [5]:
! rabbitmqadmin list exchanges

+--------------------+---------+
|        name        |  type   |
+--------------------+---------+
|                    | direct  |
| amq.direct         | direct  |
| amq.fanout         | fanout  |
| amq.headers        | headers |
| amq.match          | headers |
| amq.rabbitmq.trace | topic   |
| amq.topic          | topic   |
+--------------------+---------+


## 임시큐

이전에 기억할 수 있듯이 지정된 이름을 가진 hello 및 task_queue 큐를 사용했습니다 . 큐의 이름을 지정할 수 있다는 것이 우리에게는 매우 중요했습니다. 직원들을 동일한 큐로 안내해야 했습니다. 프로듀서와 컨슈머간에 큐를 공유하려면 큐에 이름을 지정하는 것이 중요합니다.

그러나 그것은 우리의 로거의 경우가 아닙니다. 우리는 모든 로그 메시지를 듣고 싶습니다. 우리는 또한 오래된 메시지가 아닌 현재 흐르는 메시지에만 관심이 있습니다. 이를 해결하기 위해서는 두 가지가 필요합니다.

첫째, 우리가 Rabbit에 연결할 때마다 우리는 새롭고 빈 큐가 필요합니다. 이를 위해 우리는 임의의 이름으로 큐를 생성 할 수 있습니다. 서버가 임의의 큐 이름을 선택하도록 할 수도 있습니다. queue 매개 변수를 queue_declare에 제공하지 않으면이 작업을 수행 할 수 있습니다.


In [6]:
! rabbitmqctl list_bindings

Listing bindings for vhost /...
source_name	source_kind	destination_name	destination_kind	routing_key	arguments
	exchange	task_queue	queue	task_queue	[]
	exchange	hello	queue	hello	[]


In [7]:
! rabbitmqadmin list bindings

+--------+-------------+-------------+
| source | destination | routing_key |
+--------+-------------+-------------+
|        | hello       | hello       |
|        | task_queue  | task_queue  |
+--------+-------------+-------------+


##  emit_log.py 

- 원문은 2018년에 외국 문서를 번역한 것으로써, 코드가 정확히 동작하지 않습니다.
- 원문 링크를 확인후 실행하니 원하는 방식대로 기능합니다.
- https://www.rabbitmq.com/tutorials/tutorial-three-python.htmlhttps://www.rabbitmq.com/tutorials/tutorial-three-python.html
- https://github.com/rabbitmq/rabbitmq-tutorials/blob/master/python/emit_log.pyhttps://github.com/rabbitmq/rabbitmq-tutorials/blob/master/python/emit_log.py

In [89]:
#!/usr/bin/env python
import pika
import sys

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='logs', exchange_type='fanout')

message = ' '.join(sys.argv[1:]) or "info: Hello World!"
channel.basic_publish(exchange='logs', routing_key='', body=message)
print(" [x] Sent %r" % message)
connection.close()

 [x] Sent '-f /Users/jpark/Library/Jupyter/runtime/kernel-a0e368d5-e61e-43a1-91a5-742b4475709c.json'


In [90]:
! rabbitmqadmin list exchanges

+--------------------+---------+
|        name        |  type   |
+--------------------+---------+
|                    | direct  |
| amq.direct         | direct  |
| amq.fanout         | fanout  |
| amq.headers        | headers |
| amq.match          | headers |
| amq.rabbitmq.trace | topic   |
| amq.topic          | topic   |
| logs               | fanout  |
+--------------------+---------+


In [91]:
! rabbitmqadmin list queues

+--------------------------------+----------+
|              name              | messages |
+--------------------------------+----------+
| amq.gen-UFOhCq9P19yhtbN6G98Xnw | 0        |
| amq.gen-b40As3H2aeUm8Z2f6buFxQ | 0        |
| amq.gen-oD3kH8VIyDwZkYzF95athQ | 0        |
| hello                          | 0        |
| log_queue                      | 14       |
| task_queue                     | 0        |
+--------------------------------+----------+


In [92]:
! rabbitmqadmin list bindings

+--------+--------------------------------+--------------------------------+
| source |          destination           |          routing_key           |
+--------+--------------------------------+--------------------------------+
|        | amq.gen-UFOhCq9P19yhtbN6G98Xnw | amq.gen-UFOhCq9P19yhtbN6G98Xnw |
|        | amq.gen-b40As3H2aeUm8Z2f6buFxQ | amq.gen-b40As3H2aeUm8Z2f6buFxQ |
|        | amq.gen-oD3kH8VIyDwZkYzF95athQ | amq.gen-oD3kH8VIyDwZkYzF95athQ |
|        | hello                          | hello                          |
|        | log_queue                      | log_queue                      |
|        | task_queue                     | task_queue                     |
| logs   | amq.gen-UFOhCq9P19yhtbN6G98Xnw | amq.gen-UFOhCq9P19yhtbN6G98Xnw |
| logs   | amq.gen-b40As3H2aeUm8Z2f6buFxQ | amq.gen-b40As3H2aeUm8Z2f6buFxQ |
| logs   | amq.gen-oD3kH8VIyDwZkYzF95athQ | amq.gen-oD3kH8VIyDwZkYzF95athQ |
| logs   | log_queue                      | log_queue                      |

## receive_logs.py

In [86]:
import pika, sys, os

def receive_logs():
    connection = pika.BlockingConnection(
        pika.ConnectionParameters(host='localhost'))
    channel = connection.channel()

    channel.exchange_declare(exchange='logs', exchange_type='fanout')

    result = channel.queue_declare(queue='', exclusive=True)
    queue_name = result.method.queue

    channel.queue_bind(exchange='logs', queue=queue_name)

    def callback(ch, method, properties, body):
        print(" [x] %r" % body.decode())

    print(' [*] Waiting for logs. To exit press CTRL+C')
    channel.basic_consume(
        queue=queue_name, on_message_callback=callback, auto_ack=True)

    channel.start_consuming()


if __name__ == '__main__':
    try:
        receive_logs()
    except KeyboardInterrupt:
        print('Interrupted')


 [*] Waiting for logs. To exit press CTRL+C
Interrupted


In [87]:
! rabbitmqctl list_bindings

Listing bindings for vhost /...
source_name	source_kind	destination_name	destination_kind	routing_key	arguments
	exchange	log_queue	queue	log_queue	[]
	exchange	amq.gen-b40As3H2aeUm8Z2f6buFxQ	queue	amq.gen-b40As3H2aeUm8Z2f6buFxQ	[]
	exchange	task_queue	queue	task_queue	[]
	exchange	amq.gen-UFOhCq9P19yhtbN6G98Xnw	queue	amq.gen-UFOhCq9P19yhtbN6G98Xnw	[]
	exchange	hello	queue	hello	[]
	exchange	amq.gen-oD3kH8VIyDwZkYzF95athQ	queue	amq.gen-oD3kH8VIyDwZkYzF95athQ	[]
logs	exchange	amq.gen-UFOhCq9P19yhtbN6G98Xnw	queue	amq.gen-UFOhCq9P19yhtbN6G98Xnw	[]
logs	exchange	amq.gen-b40As3H2aeUm8Z2f6buFxQ	queue	amq.gen-b40As3H2aeUm8Z2f6buFxQ	[]
logs	exchange	amq.gen-oD3kH8VIyDwZkYzF95athQ	queue	amq.gen-oD3kH8VIyDwZkYzF95athQ	[]
logs	exchange	log_queue	queue	log_queue	[]


In [88]:
! rabbitmqadmin list bindings

+--------+--------------------------------+--------------------------------+
| source |          destination           |          routing_key           |
+--------+--------------------------------+--------------------------------+
|        | amq.gen-UFOhCq9P19yhtbN6G98Xnw | amq.gen-UFOhCq9P19yhtbN6G98Xnw |
|        | amq.gen-b40As3H2aeUm8Z2f6buFxQ | amq.gen-b40As3H2aeUm8Z2f6buFxQ |
|        | amq.gen-oD3kH8VIyDwZkYzF95athQ | amq.gen-oD3kH8VIyDwZkYzF95athQ |
|        | hello                          | hello                          |
|        | log_queue                      | log_queue                      |
|        | task_queue                     | task_queue                     |
| logs   | amq.gen-UFOhCq9P19yhtbN6G98Xnw | amq.gen-UFOhCq9P19yhtbN6G98Xnw |
| logs   | amq.gen-b40As3H2aeUm8Z2f6buFxQ | amq.gen-b40As3H2aeUm8Z2f6buFxQ |
| logs   | amq.gen-oD3kH8VIyDwZkYzF95athQ | amq.gen-oD3kH8VIyDwZkYzF95athQ |
| logs   | log_queue                      | log_queue                      |